In [51]:
import pandas as pd

In [52]:
import numpy as np

In [53]:
import pandas as pd
import numpy as np
import alpaca_trade_api as alpaca
import seaborn as sns
import cred
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [54]:
api = alpaca.REST(cred.YOUR_API_KEY_ID, cred.YOUR_API_SECRET_KEY, 'https://paper-api.alpaca.markets')

In [60]:
symbol='TSLA'

In [139]:
data=api.get_barset([symbol,'VXX'],'day', limit=500).df

In [140]:
data.head()

TSLA                                        VXX  \
                             open    high     low     close    volume   open   
time                                                                           
2019-10-16 00:00:00-04:00  257.39  262.10  256.92  259.6902  12800343  21.66   
2019-10-17 00:00:00-04:00  262.50  264.78  260.17  261.9800   9112414  21.21   
2019-10-18 00:00:00-04:00  260.70  262.80  255.10  256.9899  11065826  21.38   
2019-10-21 00:00:00-04:00  258.20  259.50  250.18  253.4300   4429730  21.00   
2019-10-22 00:00:00-04:00  254.32  258.33  250.85  255.5200   4386182  20.58   

                                                             
                            high     low    close    volume  
time                                                         
2019-10-16 00:00:00-04:00  21.88  21.370  21.4150  25334312  
2019-10-17 00:00:00-04:00  21.43  20.965  21.3000  23384120  
2019-10-18 00:00:00-04:00  21.76  21.030  21.2404  29797937  
2019-10-21 00:00:00-04:00  21.08  20.660  20.6700  16093856  
2019-10-22 00:00:00-04:00  21.03  20.420  21.0000  18546935

In [141]:
def calculate_gain_loss(symbol,data):
    data[symbol, 'gain/loss'] = data[symbol]['close'].diff(periods=1)
    return data


def calculate_avg_gain_and_loss(symbol,data):
    data[symbol, 'avg_gain'] = 0
    data[symbol, 'avg_loss'] = 0
    data.loc[data.index[0],(symbol, 'gain/loss')] = 0
    for i in range(1,len(data[symbol])):
        if data.loc[data.index[i],(symbol,'gain/loss')]>=0 and data.loc[data.index[i-1],(symbol,'gain/loss')]>=0:
            data.loc[data.index[i],(symbol,'avg_gain')]= (data.loc[data.index[i],(symbol,'gain/loss')]+data.loc[data.index[i-1],(symbol,'gain/loss')])/2
            data.loc[data.index[i],(symbol,'avg_loss')]= 0
        elif data.loc[data.index[i],(symbol,'gain/loss')]<0 and data.loc[data.index[i-1],(symbol,'gain/loss')]<0:
            data.loc[data.index[i],(symbol,'avg_loss')]= (data.loc[data.index[i],(symbol,'gain/loss')]+data.loc[data.index[i-1],(symbol,'gain/loss')])/2
            data.loc[data.index[i],(symbol,'avg_gain')]= 0 
        elif data.loc[data.index[i],(symbol,'gain/loss')]<0 or data.loc[data.index[i-1],(symbol,'gain/loss')]<0:
            if data.loc[data.index[i],(symbol,'gain/loss')]>=0:
                data.loc[data.index[i],(symbol,'avg_gain')]= data[symbol,'gain/loss'][i]
                data.loc[data.index[i],(symbol,'avg_loss')]= data[symbol,'gain/loss'][i-1]
            else:
                data.loc[data.index[i],(symbol,'avg_gain')]= data[symbol,'gain/loss'][i-1]
                data.loc[data.index[i],(symbol,'avg_loss')]= data[symbol,'gain/loss'][i]
    return data

def calculate_rs(symbol,data):
    data[symbol,'RS'] = data[symbol,'avg_gain']/data[symbol,'avg_loss'].abs() 
    data = data.sort_index(axis=1)
    return data
    
def calculation_rsi(x):
    y = 100 - (100/(1+x))
    return y    

def calculate_rsi(symbol,data):
    data[symbol,'RSI'] = data[symbol,'RS'].apply(np.vectorize(calculation_rsi))
    data = data.sort_index(axis=1)
    return data

In [142]:
def calculate_kpi(symbol,data):
    data=calculate_gain_loss(symbol,data)
    data=calculate_avg_gain_and_loss(symbol,data)
    data=calculate_rs(symbol,data)
    data=calculate_rsi(symbol,data)
    return data

In [143]:
data=calculate_kpi(symbol,data)

In [144]:
def rolling_avg(symbol,data,main_mv=200,short_mv=7):
    data.loc[:,(symbol,str(main_mv)+'_mv')] = data[symbol]['close'].rolling(main_mv).mean()
    data.loc[:,(symbol,str(short_mv)+'_mv_low')] = data[symbol]['low'].rolling(short_mv).mean()
    data.loc[:,(symbol,str(short_mv)+'_mv_high')] = data[symbol]['high'].rolling(short_mv).mean()
    data.loc[:,(symbol,str(short_mv)+'_mv')] = data[symbol]['close'].rolling(short_mv).mean()
    data = data.sort_index(axis=1)
    return data

In [145]:
data=rolling_avg(symbol,data)

In [215]:
def plot_graph(data,symbol):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.09, subplot_titles=('Stock Price Movement', 'RSI - 2 Day'), 
               row_width=[0.2, 0.8],specs=[[{"secondary_y": True}],
                           [{"secondary_y": False}]])

    fig.add_trace(go.Candlestick(x=data.index,
                open=data[symbol]['open'],
                high=data[symbol]['high'],
                low=data[symbol]['low'],
                close=data[symbol]['close'],showlegend=False),row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data[symbol]['200_mv'],mode='lines',name='200-day moving avg'),1,1)
    fig.add_trace(go.Scatter(x=data.index, y=data[symbol]['7_mv'],mode='lines',name='7-day moving avg'),1,1)
    fig.add_trace(go.Scatter(x=data.index, y=data[symbol]['7_mv_high'],mode='lines',name='7-day high moving avg'),1,1)
    fig.add_trace(go.Scatter(x=data.index, y=data[symbol]['7_mv_low'],mode='lines',name='7-day low moving avg'),1,1)
    fig.add_trace(go.Scatter(x=data.index, y=data[symbol]['RSI'], showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=np.zeros(len(data.index)),line=dict(color="#000000"),showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=10*np.ones(len(data.index)),line=dict(color="#03C04A"),fill='tonexty',showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=100*np.ones(len(data.index)),line=dict(color="#000000"),showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=65*np.ones(len(data.index)),line=dict(color="#D0312D"),fill='tonexty',showlegend=False), row=2, col=1)
    #fig.add_trace(go.Scatter(x=data.index, y=data[symbol]['buy/sell'],line=dict(color="#D0312D"),showlegend=False), row=1, col=1, secondary_y=True)    
    fig.update_layout({'plot_bgcolor': "#222021", 'paper_bgcolor': "#21201f", 'legend_orientation': "h"},
                  legend=dict(y=0.9, x=0.9),
                  font=dict(color='#dedddc'), dragmode='pan', hovermode='x unified',
                  margin=dict(b=20, t=0, l=0, r=40))

    fig.update_xaxes(showgrid=False, zeroline=False, rangeslider_visible=False, showticklabels=False,
                 showspikes=True, spikemode='across', spikesnap='cursor', showline=False,
                 spikecolor="grey",spikethickness=1, spikedash='solid')

    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False,
                 showspikes=True, spikedash='solid',spikemode='across', 
                spikecolor="grey",spikesnap="cursor",spikethickness=1)
    
    fig.update_xaxes(rangeslider= {'visible':False}, row=2, col=1)
    fig.update_xaxes(showticklabels=True, row=1, col=1)
    fig.update_yaxes(showticklabels=True, row=1, col=1)
    
    fig.add_annotation(text="Sell-zone RSI > 65",
                  xref="paper", yref="paper",
                  x=0.9, y=0.12, showarrow=False)
    
    fig.add_annotation(text="Buy-zone RSI < 5",
                  xref="paper", yref="paper",
                  x=0.9, y=0, showarrow=False)

    fig.update_layout(spikedistance=10,hoverdistance=10)
    
    fig.update_layout(
    title="Stock Price Movement of TSLA for past 75 Days with Moving Average and RSI",title_x=0.1,title_y=0.97)

    fig.show()

In [216]:
plot_graph(data.tail(75),symbol)

In [148]:
data.tail()

TSLA                                      \
                               200_mv        7_mv   7_mv_high    7_mv_low   
time                                                                        
2021-10-04 00:00:00-04:00  701.830806  779.532857  791.402914  766.491443   
2021-10-05 00:00:00-04:00  702.455181  780.398571  794.618629  770.725729   
2021-10-06 00:00:00-04:00  703.042881  779.131429  792.855771  771.284300   
2021-10-07 00:00:00-04:00  703.761681  781.414286  794.192914  773.741429   
2021-10-08 00:00:00-04:00  704.488809  782.016500  794.604343  775.202857   

                                                                             \
                                 RS         RSI avg_gain avg_loss     close   
time                                                                          
2021-10-04 00:00:00-04:00       inf  100.000000    3.235   0.0000  781.5900   
2021-10-05 00:00:00-04:00  6.434343   86.548913    6.370  -0.9900  780.6000   
2021-10-06 00:00:00-04:00  1.949495   66.095890    1.930  -0.9900  782.5300   
2021-10-07 00:00:00-04:00       inf  100.000000    6.470   0.0000  793.5400   
2021-10-08 00:00:00-04:00  1.372048   57.842339   11.010  -8.0245  785.5155   

                                                                         \
                          gain/loss      high     low    open    volume   
time                                                                      
2021-10-04 00:00:00-04:00    6.3700  806.9699  776.12  796.52  28234226   
2021-10-05 00:00:00-04:00   -0.9900  797.3100  774.20  784.30  17494284   
2021-10-06 00:00:00-04:00    1.9300  786.6600  773.22  776.20  13556622   
2021-10-07 00:00:00-04:00   11.0100  805.0000  783.38  785.75  16655597   
2021-10-08 00:00:00-04:00   -8.0245  796.3800  780.91  796.00  15877370   

                             VXX                                 
                           close   high    low   open    volume  
time                                                             
2021-10-04 00:00:00-04:00  27.56  28.50  26.61  26.85  54434266  
2021-10-05 00:00:00-04:00  26.66  27.55  26.04  27.34  32041410  
2021-10-06 00:00:00-04:00  26.22  28.28  26.15  27.19  50652682  
2021-10-07 00:00:00-04:00  25.28  25.77  25.18  25.75  24886305  
2021-10-08 00:00:00-04:00  24.80  25.19  24.60  24.95  31291464

In [149]:
def suggest_buy_sell(symbol,data,date_range):
    position=0
    for i in range(date_range,0,-1):
        if data.loc[data.index[-i],(symbol,'200_mv')]<data.loc[data.index[-i],(symbol,'close')]:
            if data.loc[data.index[-i],(symbol,'7_mv_low')]<data.loc[data.index[-i],(symbol,'close')]:
                if data.loc[data.index[-i],(symbol,'RSI')]<5 and position in {0,-1}:
                    data.loc[data.index[-i],(symbol,'buy/sell')]=1
                    position=1
                
        if data.loc[data.index[-i],(symbol,'200_mv')]>data.loc[data.index[-i],(symbol,'close')] or data.loc[data.index[-i],(symbol,'7_mv_high')]>data.loc[data.index[-i],(symbol,'close')] and data.loc[data.index[-i],(symbol,'RSI')]>65 and position==1:
            data.loc[data.index[-i],(symbol,'buy/sell')] = -1
            position=-1
    data.loc[:,(symbol,'position')]=data.loc[:,(symbol,'buy/sell')]
    data.loc[:,(symbol,'position')].fillna(method='ffill',inplace=True)
    data.loc[:,(symbol,'position')].fillna(0,inplace=True)

In [150]:
suggest_buy_sell(symbol,data,500)

In [154]:
data.tail(50)

TSLA                                      \
                               200_mv        7_mv   7_mv_high    7_mv_low   
time                                                                        
2021-07-30 00:00:00-04:00  648.331413  658.206157  668.837114  644.778614   
2021-08-02 00:00:00-04:00  649.572363  666.854271  678.089971  652.464314   
2021-08-03 00:00:00-04:00  650.877263  676.359986  688.639971  661.565729   
2021-08-04 00:00:00-04:00  652.232263  683.877143  696.739986  670.397157   
2021-08-05 00:00:00-04:00  653.651262  693.855714  704.518543  682.421443   
2021-08-06 00:00:00-04:00  655.035863  701.272857  713.284271  690.740000   
2021-08-09 00:00:00-04:00  656.492012  706.468571  718.332843  698.787143   
2021-08-10 00:00:00-04:00  657.912462  709.667143  721.055700  703.484286   
2021-08-11 00:00:00-04:00  659.348588  709.407143  719.375700  704.314286   
2021-08-12 00:00:00-04:00  660.858179  711.169757  719.397129  704.084286   
2021-08-13 00:00:00-04:00  662.322224  712.079757  720.111414  704.857143   
2021-08-16 00:00:00-04:00  663.722974  708.011186  718.475714  699.855714   
2021-08-17 00:00:00-04:00  664.999074  703.292614  712.511429  692.885714   
2021-08-18 00:00:00-04:00  666.505324  699.801186  709.188571  687.774286   
2021-08-19 00:00:00-04:00  667.870124  694.584043  704.897143  682.875714   
2021-08-20 00:00:00-04:00  669.151874  690.619757  701.604271  678.517143   
2021-08-23 00:00:00-04:00  670.578224  688.331429  700.079971  675.852929   
2021-08-24 00:00:00-04:00  671.930171  687.074200  697.982829  674.181500   
2021-08-25 00:00:00-04:00  673.335621  690.625629  699.049971  678.124357   
2021-08-26 00:00:00-04:00  674.734421  695.667057  704.881400  685.092943   
2021-08-27 00:00:00-04:00  676.241121  698.875629  707.628543  689.771529   
2021-08-30 00:00:00-04:00  677.809621  707.069914  713.978543  696.220100   
2021-08-31 00:00:00-04:00  679.429771  715.022771  720.872843  703.754386   
2021-09-01 00:00:00-04:00  681.066271  719.008486  725.138571  710.971471   
2021-09-02 00:00:00-04:00  682.687771  722.424286  728.817143  714.957186   
2021-09-03 00:00:00-04:00  684.148671  725.672857  731.250000  717.842900   
2021-09-07 00:00:00-04:00  685.479171  733.042857  737.650000  723.791457   
2021-09-08 00:00:00-04:00  686.752806  739.062429  744.714286  729.417157   
2021-09-09 00:00:00-04:00  688.079906  742.495286  749.157143  734.974314   
2021-09-10 00:00:00-04:00  689.152756  742.563857  752.331429  736.128614   
2021-09-13 00:00:00-04:00  690.089056  743.855286  752.730000  732.925743   
2021-09-14 00:00:00-04:00  690.941756  745.591000  754.658571  733.762886   
2021-09-15 00:00:00-04:00  691.792356  748.751000  757.924286  735.785743   
2021-09-16 00:00:00-04:00  692.744506  749.346714  757.740000  736.978600   
2021-09-17 00:00:00-04:00  693.618806  750.227143  757.252857  738.195743   
2021-09-20 00:00:00-04:00  694.426106  746.712857  754.381429  733.480714   
2021-09-21 00:00:00-04:00  695.156606  747.161429  751.828557  732.897843   
2021-09-22 00:00:00-04:00  695.920356  748.394286  753.098543  737.222129   
2021-09-23 00:00:00-04:00  696.478756  749.700000  753.631400  738.867843   
2021-09-24 00:00:00-04:00  697.102056  752.367143  756.194257  739.753557   
2021-09-27 00:00:00-04:00  698.038206  757.294286  761.921400  742.853557   
2021-09-28 00:00:00-04:00  698.789356  759.795714  766.864257  745.165000   
2021-09-29 00:00:00-04:00  699.645856  767.087143  774.221400  752.601443   
2021-09-30 00:00:00-04:00  700.322506  772.192857  780.562914  758.967157   
2021-10-01 00:00:00-04:00  701.036206  775.545714  784.435786  762.462871   
2021-10-04 00:00:00-04:00  701.830806  779.532857  791.402914  766.491443   
2021-10-05 00:00:00-04:00  702.455181  780.398571  794.618629  770.725729   
2021-10-06 00:00:00-04:00  703.042881  779.131429  792.855771  771.284300   
2021-10-07 00:00:00-04:00  703.761681  781.414286  794.192914  773.741429   
2021-10-08 00:00:00-04:00  70

In [152]:
data[symbol]['buy/sell'].count()

201

In [ ]:
def suggest_buy_sell(symbol,data,date_range):
    for i in range(date_range,0,-1):
        if data.loc[data.index[-i],(symbol,'200_mv')]<data.loc[data.index[-i],(symbol,'close')]:
            if data.loc[data.index[-i],(symbol,'7_mv_low')]<data.loc[data.index[-i],(symbol,'close')]:
                if data.loc[data.index[-i],(symbol,'RSI')]<10:
                    data.loc[data.index[-i],(symbol,'buy/sell')]=1
                
        if data.loc[data.index[-i],(symbol,'200_mv')]>data.loc[data.index[-i],(symbol,'close')] or data.loc[data.index[-i],(symbol,'7_mv_high')]>data.loc[data.index[-i],(symbol,'close')] and data.loc[data.index[-i],(symbol,'RSI')]>65:
            data.loc[data.index[-i],(symbol,'buy/sell')] = -1